In [49]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Read in the data
demand_points = gpd.read_file('data/derived_data/kmeans_clusters.gpkg')
potential_sites = gpd.read_file('data/derived_data/all_pot_sites.gpkg')
matrix=pd.read_csv('data/derived_data/distance_matrix_walking.csv')
rcps=gpd.read_file('data/raw_data/geodata_stadt_Zuerich/recycling_sammelstellen/data/stzh.poi_sammelstelle_view.shp')

In [61]:
demand_points

,cluster_id,kmeans_cluster,total_est_pop,geometry
0,0,0,203.413468,POINT (8.55526 47.35417)
1,1,1,300.590392,POINT (8.52493 47.37948)
2,2,2,194.200746,POINT (8.55821 47.40813)
3,3,3,283.941326,POINT (8.49178 47.37514)
4,4,4,97.440936,POINT (8.50299 47.40482)
...,...,...,...,...
1995,1995,1995,360.348569,POINT (8.51951 47.37107)
1996,1996,1996,222.813028,POINT (8.52956 47.34231)
1997,1997,1997,521.064986,POINT (8.52954 47.38371)
1998,1998,1998,144.728798,POINT (8.52135 47.39693)


In [50]:
distance_matrix = matrix.pivot(index='ID', columns='cluster_ID', values='Walking_Duration_Minutes')

# get global mean and std of matrix
global_mean = distance_matrix.mean().mean()
global_std = distance_matrix.stack().std()

# generate synthetic data wiht same mean and std
size = [1200, 645]
synthetic_data = np.random.normal(global_mean, global_std, size=size)


# Get the lowest n entries for each column in synthetic_data
n = 50
for j in range(synthetic_data.shape[1]):
    column = synthetic_data[:, j]
    threshold = np.partition(column, n-1)[n-1]  # Get 15th smallest value
    mask = column > threshold
    synthetic_data[mask, j] = 10000000  # Set all values above threshold to penalty

# print number of entries above threshold
print((synthetic_data != 10000000).sum())

32250


In [70]:
distance_matrix = matrix.pivot(index='ID', columns='cluster_ID', values='Walking_Duration_Minutes')



In [79]:
distance_matrix.shape

(481, 2000)

In [52]:
# Get the lowest n entries for each column in distance_matrix
n = 400
for column in distance_matrix.columns:
    # Convert column to numpy array for efficient computation
    values = distance_matrix[column].values
    threshold = np.partition(values, n-1)[n-1]  # Get nth smallest value
    mask = values > threshold
    distance_matrix.loc[mask, column] = 10000000  # Set all values above threshold to penalty

# print number of entries below penalty value
print((distance_matrix != 10000000).sum().sum())

800007


In [ ]:
import pulp

# Parameter: number of NEW facilities to open (in addition to already open sites)
p = 12

# Get dimensions from the real distance matrix
num_demand_points = distance_matrix.shape[1]  # number of demand points
num_potential_sites = distance_matrix.shape[0]  # number of potential sites
I = range(num_demand_points)
J = range(num_potential_sites)

# Set population per demand point using total_est_pop from demand_points
pop = demand_points['total_est_pop'].to_dict()

# Identify already open sites and potential sites
existing = [j for j in J if potential_sites.iloc[j]['status'] == "open"]
new_sites = [j for j in J if potential_sites.iloc[j]['status'] == "potential"]

# Create the model
prob = pulp.LpProblem("P-Median_Problem", pulp.LpMinimize)

# Decision Variables: y only for potential new sites
y = pulp.LpVariable.dicts("Facility", new_sites, cat='Binary')

# Assignment decision variables (for all sites)
x = pulp.LpVariable.dicts("Assign",
                          [(i, j) for i in I for j in J],
                          cat='Binary')

# Objective Function: Minimize total weighted distance
prob += pulp.lpSum(pop[i] * distance_matrix.iloc[j, i] * x[i, j] for i in I for j in J)

# Constraint: Each demand point is assigned to exactly one facility.
for i in I:
    prob += pulp.lpSum(x[i, j] for j in J) == 1

# Constraint: A demand point can only be assigned to an open facility.
for i in I:
    for j in J:
        if j in new_sites:
            # For potential sites, assignment allowed only if facility is selected (y[j]==1)
            prob += x[i, j] <= y[j]
        else:
            # Existing sites are already open
            prob += x[i, j] <= 1

# Constraint: Exactly p new facilities are opened (among potential sites).
prob += pulp.lpSum(y[j] for j in new_sites) == p

# Solve the model using Gurobi
prob.solve(pulp.GUROBI(msg=True))

# Results
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Total Weighted Distance: {pulp.value(prob.objective)}")

# Get new opened facilities from potential sites
opened_new = [j for j in new_sites if pulp.value(y[j]) == 1]

# Combine existing with new opened sites for the full set of open facilities
opened_facilities = existing + opened_new

# Create a dataframe of selected sites
selected_sites = potential_sites.iloc[opened_facilities].copy()

# Assignments: create a dictionary of demand point assignments.
assignments = {(i, j): pulp.value(x[i, j]) for i in I for j in J if pulp.value(x[i, j]) == 1}


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "EndeavourOS")


INFO:gurobipy:Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "EndeavourOS")


INFO:gurobipy:


CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]


INFO:gurobipy:CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]


Thread count: 22 physical cores, 22 logical processors, using up to 22 threads


INFO:gurobipy:Thread count: 22 physical cores, 22 logical processors, using up to 22 threads


INFO:gurobipy:


In [94]:
selected_sites.to_file('data/derived_data/selected_sites_optimisation.gpkg', driver='GPKG')
objective_value = pulp.value(prob.objective)

objective_value/demand_points['total_est_pop'].sum()

3.603030643764803

In [99]:
# Calculate the total population assigned to each facility

# facility_pop will accumulate the sum of populations for each facility (j)
facility_pop = {}
for (i, j), val in assignments.items():
    facility_pop[j] = facility_pop.get(j, 0) + pop[i]

# Convert the results to a DataFrame for easier viewing
facility_pop_df = pd.DataFrame({
    'Facility_Index': list(facility_pop.keys()),
    'Assigned_Population': list(facility_pop.values())
})

# Map the facility index to the actual facility ID from potential_sites
facility_pop_df['Facility_ID'] = facility_pop_df['Facility_Index'].apply(lambda j: potential_sites.iloc[j]['ID'])

facility_pop_df

,Facility_Index,Assigned_Population,Facility_ID
0,63,2972.814485,e_64
1,140,2933.932721,e_141
2,175,2791.849183,e_176
3,45,4333.698865,e_46
4,76,1022.413927,e_77
...,...,...,...
188,118,494.071266,e_119
189,49,103.000000,e_50
190,113,50.522388,e_114
191,68,300.642503,e_69


In [100]:
facility_pop_df['Assigned_Population'].sum()

446930.6818586941

In [96]:
import folium

# Convert selected_sites to WGS 84 if not already in lat/lon
selected_sites_4326 = selected_sites.to_crs(epsg=4326)

# Set the map center to the mean coordinates of all sites
mean_lat = selected_sites_4326.geometry.y.mean()
mean_lon = selected_sites_4326.geometry.x.mean()
m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12)

# Add markers with different colors:
# - Open sites (existing) in blue
# - New sites (potential) in red
for idx, row in selected_sites_4326.iterrows():
    coords = [row.geometry.y, row.geometry.x]
    color = 'blue' if row['status'] == 'open' else 'red'
    folium.Marker(location=coords, popup=row['ID'], icon=folium.Icon(color=color)).add_to(m)

m



In [101]:
import openrouteservice
import scripts.util as util
import geopandas as gpd

# Load the flats_population dataset (produced by the population allocation step)
flats_population = gpd.read_file('data/derived_data/flats_population.gpkg').to_crs(epsg=4326)

flats_population = flats_population.groupby('egid').agg({'est_pop': 'sum', 'geometry': 'first'}).reset_index()

# Initialize ORS client (using our local ORS instance)
client = openrouteservice.Client(base_url='http://localhost:8080/ors')

selected_sites_4326 = selected_sites.to_crs(epsg=4326)
# Build the BallTree using the selected_sites_4326 as rcp locations
selected_sites_4326 = selected_sites_4326.rename(columns={'ID': 'poi_id'})
tree, rcp_coords, rcp_ids = util.initialize_ball_tree(selected_sites_4326)

# For each flat in flats_population, find the nearest RCP and its walking duration.
# It is assumed that util.find_nearest_rcp_duration returns a tuple (rcp_id, duration_minutes)
results = flats_population.apply(lambda row: util.find_nearest_rcp_duration(row.geometry, tree, rcp_coords, rcp_ids, client), axis=1)

# Extract durations (in minutes); if no valid duration is returned, assign NaN.
flats_population['duration'] = results.apply(lambda res: res[1] if res is not None and res[1] is not None else np.nan)

# Compute the weighted average walking duration using the 'est_pop' column as weights.
weighted_avg = (flats_population['duration'] * flats_population['est_pop']).sum() / flats_population['est_pop'].sum()

print("Weighted Average Walking Duration (min):", weighted_avg)

INFO:scripts.util:BallTree initialized with RCP coordinates.


Weighted Average Walking Duration (min): 3.9739535193647986


In [97]:
# write in min:sec format
minutes = int(weighted_avg)
seconds = int((weighted_avg - minutes) * 60)
print(f"Weighted Average Walking Duration: {minutes} min {seconds} sec")

Weighted Average Walking Duration: 3 min 58 sec


In [ ]:
selected_sites_4326['ID']

KeyError: 'ID'